In [31]:
__author__ = "Olivier Witteman"
__email__ = "olivier@2001.net"

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
import numpy as np
import time
import scipy.stats as sps
from scitime import Estimator
import time_complexity as tcom

In [20]:
# General parameters
n = 1

# Dataset parameters
features = 8
set_size = int(1e4)

X, y = make_regression(n_samples=set_size, n_features=features, random_state=0)

# Random forest parameters
max_trees = 100
tries_tree = set_size

# Neural network parameters
output_nodes = 1
iterations = 30
hidden_layers = (100, 100, 100, 100)
batch_size = 200

In [21]:
estimator = Estimator(meta_algo='RF', verbose=2)
rfr = RandomForestRegressor(n_estimators=max_trees)

# Estimation by scitime
estimation, lower_bound, upper_bound = estimator.time(rfr, X, y=y)

tlst_rf = []

for _ in range(n):
    t0 = time.time()
    rfr.fit(X, y)
    tlst_rf.append(time.time()-t0)

    print('\r{!s}'.format(sps.describe(tlst_rf)), end='')

scitime.estimate.Estimator:INFO:Training your model for these params: {'total_memory': [8589934592], 'available_memory': [2230190080], 'num_cpu': [4], 'num_rows': [100000], 'num_features': [8], 'n_estimators': [100], 'max_depth': [None], 'min_samples_split': [2], 'min_samples_leaf': [1], 'min_weight_fraction_leaf': [0.0], 'max_features': ['auto'], 'criterion': ['mse'], 'max_leaf_nodes': [None], 'min_impurity_decrease': [0.0], 'min_impurity_split': [None], 'bootstrap': [True], 'oob_score': [False], 'n_jobs': [None]}
scitime.estimate.Estimator:INFO:Transforming dataset for semi dummy features
scitime.estimate.Estimator:WARNING:Parameters ['ccp_alpha', 'max_samples', 'random_state', 'verbose', 'warm_start'] will not be accounted for
scitime.estimate.Estimator:INFO:Training your RandomForestRegressor model should take ~ 29 seconds
scitime.estimate.Estimator:INFO:The 95.0% prediction interval is [1 second, 1 minute
                and 16 seconds]


DescribeResult(nobs=5, minmax=(81.40264391899109, 90.14986515045166), mean=84.70289516448975, variance=18.322374730029708, skewness=0.45871045166988206, kurtosis=-1.7168310095765849)

In [22]:
# y_hat = rfr.predict(X)
# print(y.shape)
# np.mean(np.abs(y-y_hat))

In [30]:
trees = len(rfr.estimators_)
t_b = tcom.big_o_rfr(n_base=np.average(tlst_rf), n_tree=trees, m_try=tries_tree, n=set_size, inv=True)

# print(t_b)
print(tcom.big_o_nn(n_base=t_b, m=features, o=output_nodes, i=iterations, nodes=hidden_layers, t=set_size, inv=False))

5.297002936877582


In [24]:
nn = MLPRegressor(hidden_layer_sizes=hidden_layers, max_iter=iterations, batch_size=batch_size)

tlst_nn = []

for _ in range(n):
    t0 = time.time()
    nn.fit(X, y)
    tlst_nn.append(time.time()-t0)

    print('\r{!s}'.format(sps.describe(tlst_nn)), end='')

DescribeResult(nobs=5, minmax=(1.5020480155944824, 60.19721579551697), mean=28.534644317626952, variance=707.7125022920035, skewness=0.29481995347797896, kurtosis=-1.708871511477005))

In [25]:
# y_hat = nn.predict(X)
# np.mean(np.abs(y-y_hat))